I am a big fan of korean dramas. I've been watching thm since I was in junior high. With the huge variety of korean dramas it's hard for me to know which are worth my time to watch. I'm hoping that I can use ML and NLP to provide accurate predictions for kdrama fans.

The first issue I have is figuring out how to get a large database of kdramas. I do not have a ton of experience with webscraping so using some of the kdrama websites is a little hard. I decided to opt for getting a list of kdramas from wikipedia first and then cross it with imdb database to extract information there.

In [1]:
import requests
import pandas as pd

kdramaWikiPage = requests.get("https://en.wikipedia.org/wiki/List_of_South_Korean_dramas")

from bs4 import BeautifulSoup
soup = BeautifulSoup(kdramaWikiPage.text, 'lxml')
#print(soup.prettify())

In [2]:
koreanDramaNames = []
KoreanDramaYears = []

for dramaListed in soup.find_all('li'):
    subDramaListed = dramaListed.find_all('ul')
    if len(subDramaListed) > 0:
        for kDramaName in subDramaListed[0].find_all('i'):
            koreanDramaNames.append(kDramaName.text.strip())
            nameAndyear = subDramaListed[0].text
            startIndex = nameAndyear.index("(") + 1
            KoreanDramaYears.append(nameAndyear[startIndex:-1])
    else:
        allKdramas = dramaListed.find_all('i')
        if len(allKdramas) > 0:
            kDramaName = allKdramas[0].text
            if kDramaName.strip() != "":
                dramaListedText = dramaListed.text
                koreanDramaNames.append(kDramaName.strip())
                startIndex = dramaListedText.index("(") + 1
                KoreanDramaYears.append(dramaListedText[startIndex:-1])

wikiKdramas = {'title': koreanDramaNames, 'year':KoreanDramaYears}
wikiKdramasDf = pd.DataFrame(wikiKdramas)

wikiKdramasDf.head()

,title,year
0,100 Days My Prince,2018
1,12 Signs of Love,2012
2,12 Years Promise,2014
3,18 Again,2020
4,365: Repeat the Year,2020


In [3]:
wikiKdramasDf.shape

(1574, 2)

The years via the kdrama wiki page have different formats. I need a uniform start and end date. The reason for this is because I don't know what year format imdb uses. This will be extremely important for later.

In [4]:
wikiKdramasDf = wikiKdramasDf.loc[wikiKdramasDf['year'].str.len() < 13]
wikiKdramasDf['year'].unique()

array(['2018', '2012', '2014', '2020', '2014–15', '2011', '2005',
       '2016–17', '2013', '2006–07', '2016', '2015', '2007', '2017–18',
       '2013–14', '2020–21', '2010', '2022', '2019', '2018–19', '2011–12',
       '2023', '2021', '2008', '2002', '2003–04', '2000', '2015–16',
       '2003', '2010–11', '2006', '2004', '2017', '1995', '2009–10',
       '2005–06', '2004–06', '2001', '2019–20', '2009', '2021–22',
       '2012–13', 'cancelled', '2007–08', '1981', '1999', '2008–09',
       '2016–20', '2004–05', '2001–02', '1991–92', '1996–97', '1983',
       'TBA', '1999–2000', '1993', '1975–78', '2019–present', '2019–21',
       '2017–19', '2002–04', '2002–03', '1997', '2000–01', '2000–06',
       '1996', '2010–19', '1994–95', '2003–07', '2011–13', '2017–present',
       '2000–02', '1996–98', '1999–2009', '1998–2000', '1994',
       '2007–2019', '2019–2020', '1998', '1997–98'], dtype=object)

In [5]:
wikiKdramasDf = wikiKdramasDf[~wikiKdramasDf['year'].isin(['cancelled', 'TBA'])]
wikiKdramasDf.shape

(1522, 2)

In [6]:
def getEndDate(year):
    splitStartAndEnd = year.split("–")
    start = splitStartAndEnd[0]
    if len(splitStartAndEnd) > 1:
        if splitStartAndEnd[1] == "present":
            end = 2023
        else:
            end = splitStartAndEnd[1] if len(splitStartAndEnd[1]) > 2 else start[:2]+splitStartAndEnd[1]
    else:
        end = start
    return int(end)

wikiKdramasDf['endYear'] = wikiKdramasDf['year'].map(lambda year: getEndDate(year))
wikiKdramasDf['year'] = wikiKdramasDf['year'].map(lambda year: int(year.split("–")[0]))

wikiKdramasDf

,title,year,endYear
0,100 Days My Prince,2018,2018
1,12 Signs of Love,2012,2012
2,12 Years Promise,2014,2014
3,18 Again,2020,2020
4,365: Repeat the Year,2020,2020
...,...,...,...
1569,Your Scene,2007,2007
1570,Youth,2021,2021
1571,Youth of May,2021,2021
1572,Yumi's Cells,2021,2022


In [7]:
wikiKdramasDf['year'].unique()

array([2018, 2012, 2014, 2020, 2011, 2005, 2016, 2013, 2006, 2015, 2007,
       2017, 2010, 2022, 2019, 2023, 2021, 2008, 2002, 2003, 2000, 2004,
       1995, 2009, 2001, 1981, 1999, 1991, 1996, 1983, 1993, 1975, 1997,
       1994, 1998])

In [8]:
wikiKdramasDf.head()

,title,year,endYear
0,100 Days My Prince,2018,2018
1,12 Signs of Love,2012,2012
2,12 Years Promise,2014,2014
3,18 Again,2020,2020
4,365: Repeat the Year,2020,2020


In [9]:
from imdb import Cinemagoer
ia = Cinemagoer()

for idx, d in wikiKdramasDf.iterrows():
    title = d['title']
    year = d['year']
    found = []
    
    try:
        dramas = ia.search_movie_advanced(title)
        for dr in dramas:
            #Match by Year
            try:
                if (dr['title'] == title and dr['year'] == year):
                    found.append(dr)
            except:
                print(title, 'year not found')
        
        # If found > 1 drama with same name
        if len(found) == 1:
            wikiKdramasDf.loc[wikiKdramasDf['title'] == d['title'], 'movie_id'] = found[0].movieID
        elif len(found) > 1:
            wikiKdramasDf.loc[wikiKdramasDf['title'] == d['title'], 'movie_id'] = [x.movieID for x in found]

        print("Drama: ", title, "success")
    except:
        print("Drama: ", title, "error")

Drama:  100 Days My Prince success
Drama:  12 Signs of Love success
Drama:  12 Years Promise success
Drama:  18 Again success
Drama:  365: Repeat the Year success
Drama:  4 Legendary Witches success
49 Days year not found
Drama:  49 Days success
Drama:  5th Republic success
Drama:  7 First Kisses success
Drama:  7th Grade Civil Servant success
Drama:  90 Days, Time to Love success
Drama:  A Beautiful Mind success
Drama:  A Bird That Doesn't Sing success
Drama:  A Daughter Just Like You success
Drama:  A Gentleman's Dignity success
Drama:  A Girl Who Sees Smells success
Drama:  A Happy  Woman success
Drama:  A Hundred Year Legacy success
Drama:  A Korean Odyssey success
Drama:  A Little Love Never Hurts success
Drama:  A Love So Beautiful success
Drama:  A Love to Kill success
Drama:  A Man Called God success
Drama:  A Model Family success
Drama:  A New Leaf success
Drama:  A Piece of Your Mind success
Drama:  A Place in the Sun success
Drama:  A Pledge to God success
Drama:  A Poem a D

2022-10-30 23:52:28,254 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Be Positive error
Drama:  Be Strong, Geum-soon! success
Drama:  Beating Again success
Drama:  Beating Heart success
Drama:  Beautiful Days success
Drama:  Beautiful Gong Shim success
Drama:  Beautiful Love, Wonderful Life success
Drama:  Beautiful World success
Drama:  Because This Is My First Life error
Drama:  Because This Is My First Twenty success
Drama:  Becky's Back success
Drama:  Becoming a Billionaire success
Drama:  Becoming Witch success
Drama:  Beethoven Virus success
Drama:  Behind the White Tower success
Drama:  Bel Ami success
Drama:  Best Chicken success


2022-10-30 23:52:44,935 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Beyond Evil error
Beyond the Clouds year not found
Drama:  Beyond the Clouds success
Drama:  Big success
Drama:  Big Issue error
Drama:  Big Man success
Drama:  Big Mouth success
Drama:  Big Thing success
Drama:  Billie Jean, Look at Me success
Drama:  Birdie Buddy success
Drama:  Birth of a Beauty success
Drama:  Birthcare Center success
Drama:  Birthday Letter success
Drama:  Bitter Sweet Life success
Drama:  Bizarre Bunch success
Drama:  Black success
Drama:  Black Dog: Being A Teacher success
Black Knight year not found
Black Knight year not found
Drama:  Black Knight success
Drama:  Black Knight: The Man Who Guards Me success
Drama:  Blade Man success
Drama:  Blessing of the Sea success
Drama:  Blind success
Drama:  Blood success
Drama:  Blooded Palace: The War of Flowers success
Drama:  Bloody Heart success
Drama:  Bloody Romance success
Drama:  Blossom Sisters success
Drama:  Blow Breeze success
Drama:  Blue Birthday success
Drama:  Bodyguard success
Born Again year not 

...

Drama:  The President error
Drama:  The Princess' Man success
Drama:  The Producers success
Drama:  The Promise success
Drama:  The Queen of Office success
Drama:  The Queen's Classroom success
Drama:  The Rebel success
Drama:  The Red Sleeve success
Drama:  The Reputable Family success
Drama:  The Return of Hwang Geum-bok success
Drama:  The Return of Iljimae success
Drama:  The Rich Son success
The Road Home year not found
Drama:  The Road Home success
Drama:  The Road: The Tragedy of One success
Drama:  The Royal Gambler success
Drama:  The Running Mates: Human Rights success
Drama:  The Scandal success
Drama:  The Scholar Who Walks the Night success
The School Nurse Files year not found
Drama:  The School Nurse Files success
Drama:  The Second Husband success
Drama:  The Secret House success
Drama:  The Secret Life of My Secretary success
Drama:  The Secret Lovers success
Drama:  The Secret of Birth success
Drama:  The Secret of My Love success
Drama:  The Silent Sea error
Drama:  

2022-10-31 00:08:15,741 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Thirty-Nine error
Drama:  This Is My Love success
Drama:  Thousand Years of Love success
Drama:  Three Bold Siblings success
Drama:  Three Brothers success
Drama:  Queen of the Ring success
Drama:  Romance Full of Life success
Drama:  The Universe's Star success
Drama:  Three Days success
Drama:  Three Friends success
Drama:  Three Sisters success
Drama:  Thrice Married Woman success
Drama:  Through the Darkness success
Drama:  Time Between Dog and Wolf success
Drama:  Times success
Drama:  To My Beloved success
Drama:  To the Beautiful You success
Drama:  To. Jenny success
Drama:  Today's Webtoon success
Drama:  Toji, the Land success
Drama:  Tokyo Sun Shower success
Drama:  Tomorrow success
Tomorrow Boy year not found
Drama:  Tomorrow Boy success
Drama:  Tomorrow, with You success
Drama:  Top Management success
Drama:  Top Star U-back success
Drama:  Touch success
Drama:  Touch Your Heart success
Drama:  Tracer error
Drama:  Train success
Drama:  Trap success
Drama:  Tree of 

2022-10-31 00:08:46,958 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Triple error
Drama:  Trolley success
Drama:  True Beauty success
Drama:  Tunnel success
Drama:  Twelve Nights success
Drama:  Twenty-Five Twenty-One success
Twinkle Twinkle year not found
Drama:  Twinkle Twinkle success
Drama:  Two Cops success
Drama:  Two Mothers success
Drama:  Two Outs in the Ninth Inning success
Drama:  Two Weeks success
Drama:  Two Wives success
Drama:  Two Women's Room success
Drama:  Ugly Alert success
Drama:  Ugly Miss Young-ae success
Drama:  Unasked Family success
Drama:  Uncle success
Drama:  Uncontrollably Fond success
Drama:  Under the Queen's Umbrella success
Drama:  Undercover success
Drama:  Understanding of Love success
Drama:  Unframed success
Drama:  Unkind Ladies success
Drama:  Unknown Woman success
Drama:  Unlock the Boss success
Drama:  Unstoppable Marriage error
Drama:  Until the Azalea Blooms success
Drama:  Untouchable success
Vagabond year not found
Drama:  Vagabond success
Drama:  Vampire Idol success
Drama:  Vampire Prosecutor succe

In [235]:
wikiKdramasDf[wikiKdramasDf['movie_id'].isnull()]

,title,year,movie_id
1,12 Signs of Love,2012,NaN
5,4 Legendary Witches,2014,NaN
7,5th Republic,2005,NaN
8,7 First Kisses,2016,NaN
10,"90 Days, Time to Love",2006,NaN
...,...,...,...
1563,You Don't Know Women,2017,NaN
1566,You're Beautiful,2009,NaN
1567,You're Only Mine,2014,NaN
1573,Your Scene,2007,NaN


In [32]:
ia = Cinemagoer()
temp = wikiKdramasDf[wikiKdramasDf['movie_id'].isnull()]
for idx, d in temp.iterrows():
    title = d['title']
    year = d['year']
    try:
        dramas = ia.search_movie_advanced(title)
        for dr in dramas:
            try:
                if (dr['title'] == title):
                    print(title, dr['year'], year)
                    if (dr['year'] == year):
                        print("passed test", dr.movieID)
            except:
                print('error')
    except:
        print("too may values")
dr

NameError: name 'Cinemagoer' is not defined

In [181]:
wikiKdramasDf.shape

(1574, 3)

In [182]:
wikiKdramasDf.to_csv("minedKdramas.csv", index = False)

In [186]:
notok = wikiKdramasDf.loc[(wikiKdramasDf['movie_id']).isnull()]

for idx, d in notok.head().iterrows():
    try:
        title = d['title']
        print(title)
        year = d['year']
        drama_names = ia.search_movie_advanced(title)
        found = []
        for dr in drama_names:
            try:
                if (dr['year'] == year):
                    found.append(dr)
                    print(title, 'found')
            except:
                print(title, 'year not found')
        print(found)
        if len(found) > 1:
            if len(found) > 8:
                wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = found[0].movieID
            else:
                wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = ','.join([x.movieID for x in found])
            print('multiple id')
        elif len(found) == 1:
            print(found[0].movieID)
            wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = found[0].movieID
            print('single id')
        else:
            print('no id')
    except:
        print('error')


12 Signs of Love
[]
no id
4 Legendary Witches
[]
no id
49 Days
49 Days found
49 Days year not found
[<Movie id:1935066[http] title:_49 Days (2011)_>]
1935066
single id
5th Republic
[]
no id
7 First Kisses
[]
no id


In [185]:
wikiKdramasDf

,title,year,movie_id
0,100 Days My Prince,2018,8199972
1,12 Signs of Love,2012,NaN
2,12 Years Promise,2014,5476252
3,18 Again,2020,12846096
4,365: Repeat the Year,2020,12015466
...,...,...,...
1573,Your Scene,2007,NaN
1574,Youth,2021,NaN
1575,Youth of May,2021,NaN
1576,Yumi's Cells,2021,NaN


In [135]:
ia.get_movie_infoset()

['airing',
 'akas',
 'alternate versions',
 'awards',
 'connections',
 'crazy credits',
 'critic reviews',
 'episodes',
 'external reviews',
 'external sites',
 'faqs',
 'full credits',
 'goofs',
 'keywords',
 'list',
 'locations',
 'main',
 'misc sites',
 'news',
 'official sites',
 'parents guide',
 'photo sites',
 'plot',
 'quotes',
 'recommendations',
 'release dates',
 'release info',
 'reviews',
 'sound clips',
 'soundtrack',
 'synopsis',
 'taglines',
 'technical',
 'trivia',
 'tv schedule',
 'video clips',
 'vote details']

In [324]:
wikiKdramasDf[wikiKdramasDf['movie_id'].isnull()]

KeyError: 'movie_id'